## Trying to understand how to properly process data from /data2/isshamie/mito_lineage/data/processed/200403_A00953_0090_BHC23FDSXY/trim, which is amplified by MT primers and done with 5'-end sequencing with 10x genomics. 

## Libraries and files

In [1]:
from os.path import basename, join
import os
import yaml
import pandas as pd
from glob import glob
import pysam


In [2]:
params = "../parameters/trim_adaptors.yaml"
params = yaml.load(open(params), Loader=yaml.FullLoader)
params

{'samples': 'parameters/samples.tsv',
 'raw_folder': '/data2/isshamie/mito_lineage/data/raw/',
 'dataset': '200403_A00953_0090_BHC23FDSXY',
 'primers_list': '/data2/mito_lineage/BWA-Primers-MT/final_primers_with_tag.csv',
 'adapter': 'AAGCAGTGGTATCAACGCAGAGTAC',
 'MT_genes_fasta': '/data2/mito_lineage/BWA-Primers-MT/MT.genes.fasta',
 'ref_fa': '/data2/genome/human_GRCh38/cellranger/refdata-cellranger-GRCh38-3.0.0/fasta/genome.fa',
 'transcriptome_dir': 'transcriptome=/data2/genome/human_GRCh38/cellranger/refdata-cellranger-GRCh38-3.0.0/'}

In [3]:
primers = pd.read_csv(params["primers_list"])
primers

,Name,Sequence 5',Gene,Strand,MT Position,Sequence 5' with tag,Name with tag
0,gene60922_0,TGTTTATGGGGTGATGTGAG,gene60922,-,646,AAGCAGTGGTATCAACGCAGAGTACTGTTTATGGGGTGATGTGAG,tag_gene60922_0
1,gene60923_349,CTGAGCAAGAGGTGGTGAGG,gene60923,-,1251,AAGCAGTGGTATCAACGCAGAGTACCTGAGCAAGAGGTGGTGAGG,tag_gene60923_349
2,gene60923_830,TGCTGCGTGCTTGATGCTTG,gene60923,-,770,AAGCAGTGGTATCAACGCAGAGTACTGCTGCGTGCTTGATGCTTG,tag_gene60923_830
3,gene60923_499,TAGGGCTAAGCATAGTGGGG,gene60923,-,1101,AAGCAGTGGTATCAACGCAGAGTACTAGGGCTAAGCATAGTGGGG,tag_gene60923_499
4,gene60923_0,GTTCGTCCAAGTGCACTTTC,gene60923,-,1600,AAGCAGTGGTATCAACGCAGAGTACGTTCGTCCAAGTGCACTTTC,tag_gene60923_0
...,...,...,...,...,...,...,...
117,gene60956_981,GCATGGCTAGGAATAGTCCT,gene60956,-,14905,AAGCAGTGGTATCAACGCAGAGTACGCATGGCTAGGAATAGTCCT,tag_gene60956_981
118,gene60956_831,AATATAGGCCTCGCCCGATG,gene60956,-,15055,AAGCAGTGGTATCAACGCAGAGTACAATATAGGCCTCGCCCGATG,tag_gene60956_831
119,gene60956_0,AGGCCCATTTGAGTATTTTG,gene60956,-,15886,AAGCAGTGGTATCAACGCAGAGTACAGGCCCATTTGAGTATTTTG,tag_gene60956_0
120,gene60957_0,TGTCCTTGGAAAAAGGTTTT,gene60957,-,15952,AAGCAGTGGTATCAACGCAGAGTACTGTCCTTGGAAAAAGGTTTT,tag_gene60957_0


## Create the fasta sequences:
To do this, add primer as a fasta line. Additionally, add an extended sequence of it (either to the max of the gene or extra 200 bp)  
Caveat: This may lead to duplications  


## Input: 
- MT gene fasta
- Primer sequence, position and strand (in primers_list)


In [6]:
from Bio import SeqIO

In [9]:
MT_genes = SeqIO.to_dict(SeqIO.parse(params['MT_genes_fasta'],format='fasta'))
MT_genes


{'id1959760::NC_012920.1:0-16569(+)': SeqRecord(seq=Seq('GATCACAGGTCTATCACCCTATTAACCACTCACGGGAGCTCTCCATGCATTTGG...ATG', SingleLetterAlphabet()), id='id1959760::NC_012920.1:0-16569(+)', name='id1959760::NC_012920.1:0-16569(+)', description='id1959760::NC_012920.1:0-16569(+)', dbxrefs=[]),
 'gene60922::NC_012920.1:576-647(+)': SeqRecord(seq=Seq('GTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTC...ACA', SingleLetterAlphabet()), id='gene60922::NC_012920.1:576-647(+)', name='gene60922::NC_012920.1:576-647(+)', description='gene60922::NC_012920.1:576-647(+)', dbxrefs=[]),
 'id1959761::NC_012920.1:576-647(+)': SeqRecord(seq=Seq('GTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTC...ACA', SingleLetterAlphabet()), id='id1959761::NC_012920.1:576-647(+)', name='id1959761::NC_012920.1:576-647(+)', description='id1959761::NC_012920.1:576-647(+)', dbxrefs=[]),
 'rna171173::NC_012920.1:576-647(+)': SeqRecord(seq=Seq('GTTTATGTAGCTTACCTCCTCAAAGCAATACACTGAAAATGTTTAGACGGGCTC...ACA', SingleLetterA

In [ ]:
cmd = "cellranger mkref"